In [ ]:
from selenium.webdriver.common.by import By
from find.api import APIFinder
from _nytimes import Puzzle

In [ ]:
browser = Puzzle("games/strands")
finder = APIFinder(lang="en-us")

In [ ]:
letters = [ l.text for l in browser.find_elements(By.XPATH, "//form//button[contains(@id, 'button-')]") ]
letters = [ letters[l : l + 6] for l in range(0, len(letters), 6) ]

In [ ]:
browser.end()

In [ ]:
def get_neighbors(currentR:int, currentC:int, totalR:int, totalC:int) -> list:

    return [ (newR, newC) for nextR, nextC in [(-1, -1), (-1, 0), (-1, 1), ( 0, -1), ( 0, 1), ( 1, -1), ( 1, 0), ( 1, 1)]
        if 0 <= (newR := currentR + nextR) < totalR and 0 <= (newC := currentC + nextC) < totalC ]

def deep_first_search(size:int, matrix:list[list[str]], row:int, column:int, path:list, visited:dict, found) -> str:
    if len(path) == size:
        found.add("".join(path))
        return

    for neighborR, neighborC in get_neighbors(row, column, len(matrix), len(matrix[0])):
        if (neighborR, neighborC) not in visited:

            visited.add((neighborR, neighborC))
            deep_first_search(size, matrix, neighborR, neighborC, path + [matrix[neighborR][neighborC]], visited, found)
            visited.remove((neighborR, neighborC))

def find_sequences(matrix:list[list[str]], size:int) -> set:
    [ deep_first_search(size, matrix, r, c, [matrix[r][c]], {(r, c)}, s:=set()) for c in range(len(matrix[0])) for r in range(len(matrix)) ]

    return s

In [ ]:
for size in [4, 5, 6, 7, 8]:
    finder.kwargs.WORD_LENGTH = size

    for sequence in find_sequences(letters, size):
        finder.kwargs.SEQUENCE = sequence.lower()

        if words := finder.filter():
            print(sequence, words)